In [ ]:
# Determine for each gene how expression correlates with differentiation efficiency
# (see Jerber paper fig3)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats
import statsmodels.stats.multitest as sm
import os.path

In [ ]:
# Setup

# Input dataset
expression_data_file = 'salmon.merged.gene_tpm.log2_tmp_plus_1.retention_group_filtered.tsv.gz'

# Metadata
metadata_file = 'dataset_summary.tsv'

# FDR
fdr_threshold = 0.05

# TPM
log2_tpm_threshold = 0.1

In [ ]:
# Read in data
print("Reading in metadata: " + metadata_file)
metadata = pd.read_csv(metadata_file, sep="\t")

print("Reading in: " + expression_data_file)
expression_data = pd.read_csv(expression_data_file, sep="\t")
print(f'Number of cell lines: {expression_data.shape[1] - 2}')
print(f'Number of genes: {expression_data.shape[0]}')

In [ ]:
# Identify the appropropiate differentiation efficiencies
accessions_to_select = expression_data.columns[2:]    #Ignore first 2 columns

differentiation_efficiency = (metadata
                                .loc[:, ['Accession', 'Diff_efficiency']]
                                .set_index(keys='Accession', drop=True)
                                .loc[accessions_to_select, :]   # Select in order 
                             )

In [ ]:
# Plot scatter plot for individual gene
#target_gene_id = 'ENSG00000000003'
target_gene_id = 'ENSG00000171794'
#target_gene_id = 'ENSG00000166863'

target_gene_expression_data = (expression_data
                                .query('gene_id == @target_gene_id')
                                .iloc[:, 2:]   # Remove gene id, name
                                .transpose()                        
                              )

target_gene_expression_data.columns = ['Expression']

expression_differentiation = target_gene_expression_data.join(differentiation_efficiency)
exp_diff_correlation = (expression_differentiation
                        .corr(method='pearson')
                        .iloc[1,0]
                       )

(exp_diff_correlation, 
 exp_diff_correlation_pvalue) = scipy.stats.pearsonr(
                                    expression_differentiation['Expression'],
                                    expression_differentiation['Diff_efficiency']
                                )

gene_name = (expression_data
                .query('gene_id == @target_gene_id')
                .loc[:, 'gene_name']
                .iloc[0]
            )

plt.figure(figsize=(7,7))
sns.regplot(x="Diff_efficiency", y="Expression", data=expression_differentiation)

plt.title(f'{gene_name} ({target_gene_id})\nR: {round(exp_diff_correlation,3)}\np: {round(exp_diff_correlation_pvalue,3)}')
plt.xlabel('Differentiation Efficiency')
plt.ylabel('Expression Log2(TMP+1)')
plt.xlim(0, 1)
#plt.ylim(0, 10)
plt.show()

In [ ]:
# Log2 tpm histogram
plot_data = (expression_data
                .iloc[:, 2:]
                .to_numpy()
                .flatten()
            )

plt.figure(figsize=(7,7))
plt.hist(plot_data, bins=100)
plt.xlabel('Log2(TPM + 1)')
plt.ylabel('Frequency')
plt.axvline(log2_tpm_threshold, color='r', linestyle='--')
plt.show()

In [ ]:
# Cumulative distribution plot
plt.figure(figsize=(7,7))
plot=sns.ecdfplot(data=expression_data.iloc[:, 2:], legend=False)
plt.xlabel('Log2(TPM + 1)')
plt.axvline(log2_tpm_threshold, color='r', linestyle='--')
plt.show()

In [ ]:
# Apply filtering of expression values
print(f'Number of genes before filtering: {expression_data.shape[0]}')
to_select_boolean = expression_data.iloc[:, 2:].max(axis=1)>= log2_tpm_threshold
expression_data = expression_data[to_select_boolean]
print(f'Number of genes after filtering: {expression_data.shape[0]}')


In [ ]:
# Log2 tpm histogram after filtering
plot_data = (expression_data
                .iloc[:, 2:]
                .to_numpy()
                .flatten()
            )

plt.figure(figsize=(7,7))
plt.hist(plot_data, bins=100)
plt.xlabel('Log2(TPM + 1)')
plt.ylabel('Frequency')
plt.axvline(log2_tpm_threshold, color='r', linestyle='--')
plt.show()

In [ ]:
# Cumulative distribution plot after filtering
plt.figure(figsize=(7,7))
plot=sns.ecdfplot(data=expression_data.iloc[:, 2:], legend=False)
plt.xlabel('Log2(TPM + 1)')
plt.axvline(log2_tpm_threshold, color='r', linestyle='--')
plt.show()

In [ ]:
# Determine correlation and p-values for all genes

column_names = ['target_gene_id', 'gene_name', 'R', 'p']
correlation_results = pd.DataFrame(columns = column_names)

for target_gene_id in expression_data['gene_id']:
    #print(target_gene_id)

    target_gene_expression_data = (expression_data
                                    .query('gene_id == @target_gene_id')                      
                                  )

    gene_name = (target_gene_expression_data
                    .loc[:, 'gene_name']
                    .iloc[0]
                )

    target_gene_expression_data = (target_gene_expression_data
                                    .iloc[:, 2:]   # Remove gene id, name
                                    .transpose()                        
                                  )

    target_gene_expression_data.columns = ['Expression']

    expression_differentiation = target_gene_expression_data.join(differentiation_efficiency)
    exp_diff_correlation = (expression_differentiation
                            .corr(method='pearson')
                            .iloc[1,0]
                           )

    (exp_diff_correlation, 
     exp_diff_correlation_pvalue) = scipy.stats.pearsonr(
                                        expression_differentiation['Expression'],
                                        expression_differentiation['Diff_efficiency']
                                    )

    result_to_append = pd.DataFrame([
                                    target_gene_id, 
                                    gene_name, 
                                    exp_diff_correlation, 
                                    exp_diff_correlation_pvalue]
                                 )

    result_to_append = result_to_append.transpose()
    result_to_append.columns = column_names

    correlation_results = pd.concat([correlation_results, result_to_append])

correlation_results = correlation_results.reset_index(drop=True)

In [ ]:
# Calculate Benjamini-Hochberg
p_values = (correlation_results['p']
                .to_numpy()
                .astype(float)
           )

q_values = sm.multipletests(p_values, 
                            alpha=0.05, 
                            method='fdr_bh', 
                            is_sorted=False, 
                            returnsorted=False)

q_values = q_values[1]  # Get list of q-values from tuple
correlation_results['q'] = q_values

In [ ]:
# Plot a histogram of the results

# Determine R when fdr = threshold
min_r = (correlation_results
                .loc[:, ['R', 'q']]
                .query('R < 0')
                .query('q <= @fdr_threshold')
                .loc[:, ['R']]
                .max()
                .iloc[0]
        )

max_r = (correlation_results
                .loc[:, ['R', 'q']]
                .query('R > 0')
                .query('q <= @fdr_threshold')
                .loc[:, ['R']]
                .min() 
                .iloc[0]
        )

#Plot
plt.figure(figsize=(7,7))
sns.histplot(data=correlation_results, x='R')
plt.axvline(min_r, color='r', linestyle='--')
plt.axvline(max_r, color='r', linestyle='--')
plt.show()

In [ ]:
# Write out results
outfile = os.path.basename(expression_data_file)
outfile = outfile.replace('.tsv.gz', '')
outfile = f'{outfile}.correlation.tsv.gz'
print(f'Writing results to: {outfile}')
correlation_results.to_csv(outfile, index=False, compression='gzip', sep="\t")

In [ ]:
print('Done')